## LA NACIONAL LICORES

#### Libraries 

In [97]:
pip install pymupdf pdfplumber tabula-py camelot-py[cv] pytesseract Pillow pandas camelot

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [98]:
# Lectura de pdf
import fitz  
import pdfplumber
import tabula
import pytesseract
from PIL import Image
import pandas as pd
import io
import re
import pandas as pd

#### Funciones

In [99]:
#funcion de limpieza de columnas : 
def limpiar_columnas(df):
    df.columns = (
        df.columns
        .str.strip()                      
        .str.lower()                     
        .str.replace(' ', '_')        
        .str.replace(r'[^\w\s]', '', regex=True)  
    )
    return df

In [100]:


class PDFExtractorAdaptativo:
    def __init__(self):
        self.metodos_texto = ['pdfplumber', 'pymupdf', 'ocr']
        self.metodos_tablas = ['tabula', 'pdfplumber']  # Quitamos 'camelot'
    
    def detectar_tipo_contenido(self, archivo_pdf):
        """Detecta qué tipo de contenido tiene el PDF"""
        try:
            doc = fitz.open(archivo_pdf)
            primera_pagina = doc[0]
            
            texto = primera_pagina.get_text().strip()
            imagenes = primera_pagina.get_images()
            
            with pdfplumber.open(archivo_pdf) as pdf:
                page = pdf.pages[0]
                tablas_detectadas = page.find_tables()
            
            resultado = {
                'tiene_texto': len(texto) > 50,
                'tiene_imagenes': len(imagenes) > 0,
                'posibles_tablas': len(tablas_detectadas) > 0,
                'es_escaneado': len(texto) < 50 and len(imagenes) > 0,
                'calidad_texto': 'alta' if len(texto) > 200 else 'baja'
            }
            
            doc.close()
            return resultado
            
        except Exception as e:
            return {'error': str(e)}
    
    def extraer_texto_robusto(self, archivo_pdf):
        """Extrae texto probando diferentes métodos"""
        metodos_resultados = {}
        
        # pdfplumber
        try:
            with pdfplumber.open(archivo_pdf) as pdf:
                texto = ""
                for page in pdf.pages:
                    texto += page.extract_text() + "\n"
                metodos_resultados['pdfplumber'] = texto
        except Exception as e:
            metodos_resultados['pdfplumber'] = f"Error: {e}"
        
        # pymupdf
        try:
            doc = fitz.open(archivo_pdf)
            texto = ""
            for page in doc:
                texto += page.get_text() + "\n"
            doc.close()
            metodos_resultados['pymupdf'] = texto
        except Exception as e:
            metodos_resultados['pymupdf'] = f"Error: {e}"
        
        # OCR
        try:
            doc = fitz.open(archivo_pdf)
            texto = ""
            for page_num in range(len(doc)):
                page = doc.load_page(page_num)
                pix = page.get_pixmap()
                img_data = pix.tobytes("png")
                img = Image.open(io.BytesIO(img_data))
                texto += pytesseract.image_to_string(img, lang='spa') + "\n"
            doc.close()
            metodos_resultados['ocr'] = texto
        except Exception as e:
            metodos_resultados['ocr'] = f"Error: {e}"
        
        # Seleccionar mejor resultado
        mejor_resultado = ""
        max_longitud = 0
        for metodo, resultado in metodos_resultados.items():
            if not resultado.startswith("Error") and len(resultado) > max_longitud:
                mejor_resultado = resultado
                max_longitud = len(resultado)
        
        return {
            'texto_final': mejor_resultado,
            'metodos_intentados': metodos_resultados,
            'metodo_exitoso': max([k for k, v in metodos_resultados.items() 
                                 if not v.startswith("Error") and len(v) == max_longitud], 
                                default="ninguno")
        }
    
    def extraer_tablas_robusto(self, archivo_pdf):
        """Extrae tablas probando diferentes métodos (sin camelot)"""
        resultados_tablas = {}
        
        # tabula-py
        try:
            tablas = tabula.read_pdf(archivo_pdf, pages='all', multiple_tables=True)
            resultados_tablas['tabula'] = {
                'tablas': tablas,
                'cantidad': len(tablas),
                'exito': True
            }
        except Exception as e:
            resultados_tablas['tabula'] = {'error': str(e), 'exito': False}
        
        # pdfplumber
        try:
            with pdfplumber.open(archivo_pdf) as pdf:
                todas_tablas = []
                for page in pdf.pages:
                    tablas_pagina = page.extract_tables()
                    for tabla in tablas_pagina:
                        df = pd.DataFrame(tabla[1:], columns=tabla[0])
                        todas_tablas.append(df)
                
                resultados_tablas['pdfplumber'] = {
                    'tablas': todas_tablas,
                    'cantidad': len(todas_tablas),
                    'exito': True
                }
        except Exception as e:
            resultados_tablas['pdfplumber'] = {'error': str(e), 'exito': False}
        
        return resultados_tablas
    
    def procesar_pdf_completo(self, archivo_pdf):
        """Proceso completo adaptativo"""
        print(f"Analizando: {archivo_pdf}")
        
        info_contenido = self.detectar_tipo_contenido(archivo_pdf)
        print(f"Análisis de contenido: {info_contenido}")
        
        resultado_final = {
            'archivo': archivo_pdf,
            'analisis_contenido': info_contenido,
            'texto': None,
            'tablas': None
        }
        
        if info_contenido.get('tiene_texto') or info_contenido.get('es_escaneado'):
            print("Extrayendo texto...")
            resultado_texto = self.extraer_texto_robusto(archivo_pdf)
            resultado_final['texto'] = resultado_texto
        
        if info_contenido.get('posibles_tablas'):
            print("Extrayendo tablas...")
            resultado_tablas = self.extraer_tablas_robusto(archivo_pdf)
            resultado_final['tablas'] = resultado_tablas
        
        return resultado_final

# Uso
if __name__ == "__main__":
    extractor = PDFExtractorAdaptativo()
    resultado = extractor.procesar_pdf_completo("resources/LA_NACIONAL_LICORES.pdf")


Analizando: resources/LA_NACIONAL_LICORES.pdf


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Análisis de contenido: {'tiene_texto': True, 'tiene_imagenes': True, 'posibles_tablas': True, 'es_escaneado': False, 'calidad_texto': 'alta'}
Extrayendo texto...
Extrayendo tablas...


CropBox missing from /Page, defaulting to MediaBox


In [101]:
#extraer del diccionario final tablas y texto
textos = resultado.get('texto', {}).get('texto_final', '')
print(type(textos))

#Escritura .txt
with open("LA_NACIONAL_LICORES.txt", "w", encoding="utf-8") as f:
    f.write(textos)

<class 'str'>


In [102]:

texto = textos 

# Información de la empresa
info_general = {
    "nit_empresa": re.search(r"Nit:\s*([\d\s-]+)\s*Regimen", texto).group(1).replace(" ", ""),
    "regimen": "Comun",
    "direccion_empresa": re.search(r"Av\. Americas #([\d\-]+)", texto).group(0),
    "telefono": re.search(r"\(601\)\s*\d{3}-\d{4}", texto).group(0),
    "celular_empresa": re.search(r"Cell:\s*\((\d+)\)\s*(\d+-\d+)", texto).group(0),
    "email": re.search(r"Email:\s*([^\s]+)", texto).group(1),
    "sitio_web": re.search(r"(www\.[^\s]+)", texto).group(1),
    "razon_social_cliente": re.search(r"Razon Social:\s*(\w.+?)\s+Nit:", texto, re.DOTALL).group(1).strip(),
    "nit_cliente": re.search(r"Nit:\s*(\d+)", texto).group(1),
    "direccion_cliente": re.search(r"Direccion\s*:\s*(CL .+?)\s+Celular", texto).group(1),
    "celular_vendedor": re.search(r"Celular del Vendedor\s*:\s*(\d+)", texto).group(1),
    "firma": re.search(r"Firma:\s*(.+?)\d{10}", texto).group(1).strip(),
    "negociacion_id": re.search(r"Negociacion ID:\s*([\d\-: ]+)", texto).group(1),
}


df_general = pd.DataFrame([info_general])

In [103]:
info_general

{'nit_empresa': '900378088-5',
 'regimen': 'Comun',
 'direccion_empresa': 'Av. Americas #42-66',
 'telefono': '(601) 432-5610',
 'celular_empresa': 'Cell: (311) 474-4237',
 'email': 'info@lanacionaldelicores.com',
 'sitio_web': 'www.LanacionalDeLicores.com',
 'razon_social_cliente': 'CROC SAS',
 'nit_cliente': '900',
 'direccion_cliente': 'CL 19 69 36',
 'celular_vendedor': '3508759095',
 'firma': 'Laura Guerrero',
 'negociacion_id': '2025-05-27 10:16:17'}

In [104]:
# Tabla de Productos
patron_productos = re.compile(
    r"(?P<cantidad>\d+)\s+(?P<codigo>\d+)\s+(?P<descripcion1>[^\n]+)\n(?P<descripcion2>[^\n]+)\n\$?(?P<baseu>[\d,]+)\n\$?(?P<ivau>[\d,]+)\n\$?(?P<impu>[\d,]+)\n\$?(?P<total>[\d,]+)",
    re.MULTILINE
)

productos = []
for m in patron_productos.finditer(texto):
    productos.append({
        "cantidad": int(m.group("cantidad")),
        "codigo": m.group("codigo"),
        "descripcion": (m.group("descripcion1") + " " + m.group("descripcion2")).strip(),
        "base_u": m.group("baseu").replace(",", ""),
        "iva_u": m.group("ivau").replace(",", ""),
        "imp_u": m.group("impu").replace(",", ""),
        "total": m.group("total").replace(",", ""),
    })

df_productos = pd.DataFrame(productos)

# 3. Valor de negociación (letras)
valor_negociacion = re.search(r"Valor De Negociacion\.\s*\$ (.+)", texto)
valor_letras = valor_negociacion.group(1).strip() if valor_negociacion else None



In [105]:
productos

[{'cantidad': 30,
  'codigo': '1870',
  'descripcion': 'DON CHICHARRON PICANTE  LONCHERA CJ*60*35gr',
  'base_u': '2993',
  'iva_u': '149',
  'imp_u': '598',
  'total': '112200'},
 {'cantidad': 30,
  'codigo': '10',
  'descripcion': '1891 1U DON CHICHARRON  NATURAL LONCHERA CJ*6*35gr',
  'base_u': '2993',
  'iva_u': '149',
  'imp_u': '598',
  'total': '37400'},
 {'cantidad': 10,
  'codigo': '20',
  'descripcion': '9873 DON CHICHARRON LIMON  LONCHERA CJ*60*35gr',
  'base_u': '2993',
  'iva_u': '149',
  'imp_u': '598',
  'total': '74800'},
 {'cantidad': 20,
  'codigo': '24',
  'descripcion': '3258 DON CHICHARRON FAMILIAR  LIMON CJ*1*100gr',
  'base_u': '8505',
  'iva_u': '425',
  'imp_u': '1701',
  'total': '255144'},
 {'cantidad': 24,
  'codigo': '24',
  'descripcion': '6452 DON CHICHARRON FAMILIAR  PICANTE CJ*1*100gr',
  'base_u': '8505',
  'iva_u': '425',
  'imp_u': '1701',
  'total': '255144'}]

In [106]:
df_productos

,cantidad,codigo,descripcion,base_u,iva_u,imp_u,total
0,30,1870,DON CHICHARRON PICANTE LONCHERA CJ*60*35gr,2993,149,598,112200
1,30,10,1891 1U DON CHICHARRON NATURAL LONCHERA CJ*6*...,2993,149,598,37400
2,10,20,9873 DON CHICHARRON LIMON LONCHERA CJ*60*35gr,2993,149,598,74800
3,20,24,3258 DON CHICHARRON FAMILIAR LIMON CJ*1*100gr,8505,425,1701,255144
4,24,24,6452 DON CHICHARRON FAMILIAR PICANTE CJ*1*100gr,8505,425,1701,255144


In [107]:
df_general

,nit_empresa,regimen,direccion_empresa,telefono,celular_empresa,email,sitio_web,razon_social_cliente,nit_cliente,direccion_cliente,celular_vendedor,firma,negociacion_id
0,900378088-5,Comun,Av. Americas #42-66,(601) 432-5610,Cell: (311) 474-4237,info@lanacionaldelicores.com,www.LanacionalDeLicores.com,CROC SAS,900,CL 19 69 36,3508759095,Laura Guerrero,2025-05-27 10:16:17


### Df general - Escoger Columnas -Ordenar dataframe
1. Cliente
2. Direccion
3. Destino
4. Numero de Pedido
5. Fecha de Orden
6. Fecha de Entrega
7. Id Falso (Para Cruce de Orden de compra )

In [108]:
#Creacion de False Columns
df_general['CLIENTE'] = 'LA NACIONAL LICORES S.A.S'

#Direccion y destino 
df_general['DIRECCION'] = df_general['direccion_empresa']

#Direccion y destino 
df_general['DESTINO'] = df_general['direccion_empresa']

#Numero de Pedido  -> 'Negociacion ID'
df_general['NUMERO_DE_PEDIDO'] = "1"

#Fecha de Pedido
df_general['FECHA_DE_ORDEN'] = df_general['negociacion_id']

df_general['FECHA_DE_ORDEN'] = pd.to_datetime(df_general['FECHA_DE_ORDEN']).dt.date

#Plus 5 Days to Fecha de Pedido
df_general['FECHA_DE_ENTREGA'] = df_general['FECHA_DE_ORDEN'] + pd.Timedelta(days=8)

#Crear un id false 
df_general['id'] = 1

#Select Columns 
df_general = df_general[['id', 'CLIENTE', 'DIRECCION', 'DESTINO', 'NUMERO_DE_PEDIDO', 'FECHA_DE_ORDEN', 'FECHA_DE_ENTREGA']]

In [109]:
df_general

,id,CLIENTE,DIRECCION,DESTINO,NUMERO_DE_PEDIDO,FECHA_DE_ORDEN,FECHA_DE_ENTREGA
0,1,LA NACIONAL LICORES S.A.S,Av. Americas #42-66,Av. Americas #42-66,1,2025-05-27,2025-06-04


#### Escoger Columnas Productos 
1. Numero de item
2. descripcion
3. Cantidad
4. Unidad de medida
5. Costo Unitario
6. importe
7. id falso (Unir para realizar insersion de datos)

In [110]:
df_productos["NUMERO_DE_ITEM"] = df_productos["codigo"]
df_productos["DESCRIPCION"] = df_productos["descripcion"]
df_productos["CANTIDAD"] = df_productos["cantidad"]
df_productos["UNIDAD_DE_MEDIDA"] = "UNIDAD"
df_productos["COSTO_UNITARIO"] = df_productos["base_u"]
df_productos["IMPORTE"] = df_productos["total"]
#Create id column
df_productos['id'] = 1

#Select Columns
df_productos = df_productos[['id', 'NUMERO_DE_ITEM', 'DESCRIPCION', 'CANTIDAD', 'UNIDAD_DE_MEDIDA', 'COSTO_UNITARIO', 'IMPORTE']]

### Join

In [111]:
#Join 
df_join = df_general.merge(df_productos, on='id', how='left')

#Drop Duplicates 
df_join = df_join.drop_duplicates()

In [112]:
df_join

,id,CLIENTE,DIRECCION,DESTINO,NUMERO_DE_PEDIDO,FECHA_DE_ORDEN,FECHA_DE_ENTREGA,NUMERO_DE_ITEM,DESCRIPCION,CANTIDAD,UNIDAD_DE_MEDIDA,COSTO_UNITARIO,IMPORTE
0,1,LA NACIONAL LICORES S.A.S,Av. Americas #42-66,Av. Americas #42-66,1,2025-05-27,2025-06-04,1870,DON CHICHARRON PICANTE LONCHERA CJ*60*35gr,30,UNIDAD,2993,112200
1,1,LA NACIONAL LICORES S.A.S,Av. Americas #42-66,Av. Americas #42-66,1,2025-05-27,2025-06-04,10,1891 1U DON CHICHARRON NATURAL LONCHERA CJ*6*...,30,UNIDAD,2993,37400
2,1,LA NACIONAL LICORES S.A.S,Av. Americas #42-66,Av. Americas #42-66,1,2025-05-27,2025-06-04,20,9873 DON CHICHARRON LIMON LONCHERA CJ*60*35gr,10,UNIDAD,2993,74800
3,1,LA NACIONAL LICORES S.A.S,Av. Americas #42-66,Av. Americas #42-66,1,2025-05-27,2025-06-04,24,3258 DON CHICHARRON FAMILIAR LIMON CJ*1*100gr,20,UNIDAD,8505,255144
4,1,LA NACIONAL LICORES S.A.S,Av. Americas #42-66,Av. Americas #42-66,1,2025-05-27,2025-06-04,24,6452 DON CHICHARRON FAMILIAR PICANTE CJ*1*100gr,24,UNIDAD,8505,255144


In [ ]:
#Destino Bogota, Cruzar con Codigos , cantidad revisar, Bodega ->Descripcion ->Identificar la bodega  

In [114]:
"""

----Ordenes de compra ->Crear correo 



---->Compañía industrial de cereales 

Nombre de la empresa
Fecha de comprobante
Fecha de entrega 
Orden de compra
Asociar ref -> Ean->Descripción del producto 
cantidad
Valor unitario ->Total Bruto  

----AXIONLOG

{'Fecha Orden': '29/04/2025',
 'Nro Nota Pedido': '655664',
 'Condiciones Pago': '45 Días Fecha Factura',
 'Última Fecha Entrega': '08/05/2025',--->
 'Destino': '0101 Axis Log. SAS-BOG.'}
Fecha de entrega -> Fecha Helisa 


->Productos  Kabano -> Multiplicar cantidad por 10 y valor unitario dividido en 10 


->Megamarket ->No hay que hacer modificaciones 



->Entremes -> Carnudo -> Tiene un valor diferente al sistema 

-> BYB 80 * 30 Precio unitario y Cajas 





-------------------------------------------------------------------------------



1. Mejorar Filtros
2. Ingresar Lotes > Modulo inventarios > Lote -> Unión 
3. Fecha mínima y máxima de ordenes de compra
4. Ciudad ->Despacho de orden de compra 
5. Agrupar los datos -> Cuadrar esa visual ->Ok





"""


"\n\n----Ordenes de compra ->Crear correo \n\n\n\n---->Compañía industrial de cereales \n\nNombre de la empresa\nFecha de comprobante\nFecha de entrega \nOrden de compra\nAsociar ref -> Ean->Descripción del producto \ncantidad\nValor unitario ->Total Bruto  \n\n----AXIONLOG\n\n{'Fecha Orden': '29/04/2025',\n 'Nro Nota Pedido': '655664',\n 'Condiciones Pago': '45 Días Fecha Factura',\n 'Última Fecha Entrega': '08/05/2025',--->\n 'Destino': '0101 Axis Log. SAS-BOG.'}\nFecha de entrega -> Fecha Helisa \n\n\n->Productos  Kabano -> Multiplicar cantidad por 10 y valor unitario dividido en 10 \n\n\n->Megamarket ->No hay que hacer modificaciones \n\n\n\n->Entremes -> Carnudo -> Tiene un valor diferente al sistema \n\n-> BYB 80 * 30 Precio unitario y Cajas \n\n\n\n\n\n-------------------------------------------------------------------------------\n\n\n\n1. Mejorar Filtros\n2. Ingresar Lotes > Modulo inventarios > Lote -> Unión \n3. Fecha mínima y máxima de ordenes de compra\n4. Ciudad ->Despach